In [1]:
import os
PATH=os.path.join(os.getcwd(),'master-shell-352711-035a6f0dc8c6.json')
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]=PATH

In [8]:
from google.cloud import storage
from google.cloud.storage import Blob

import io
import os 
from google.cloud.storage import Blob
import google.auth
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from google.oauth2 import service_account
from googleapiclient.http import MediaIoBaseDownload
import gspread
import pandas as pd
cred_file = "master-shell-352711-9f095697259f.json"

# Create parenet folder and subfolder form spreadsheets 

In [15]:
from __future__ import print_function
import os.path

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

# If modifying these scopes, delete the file token.json.
SCOPES = ['https://www.googleapis.com/auth/drive']
#chanel_name=content_name[0]
#     chanel_name=chanel_name.lower()
#     print(chanel_name)chanel_name=content_name[0]
#     chanel_name=chanel_name.lower()
#     print(chanel_name)

def spreadsheet_record(spreadsheetname):
    gc = gspread.service_account(cred_file)
    database = gc.open(spreadsheetname)
    wks = database.worksheet("Sheet1")
    data = pd.DataFrame(wks.get_all_records())
    creatores_name=data['Name'].to_list()
    return creatores_name



def content_details():
    gc = gspread.service_account(cred_file)
    database = gc.open("Content_details")
    wks = database.worksheet("Sheet1")
    data = pd.DataFrame(wks.get_all_records())
    content_name=data['Content_name'].to_list()
    return content_name
    


def create_parent_folder(foldername):
    
    """Shows basic usage of the Drive v3 API.
    Prints the names and ids of the first 10 files the user has access to.
    """
    creds = None
    # The file token.json stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open('token.json', 'w') as token:
            token.write(creds.to_json())

        try:
            
            # create drive api client
            service = build('drive', 'v3', credentials=creds)
            file_metadata = {
                'name': foldername,
                'mimeType': 'application/vnd.google-apps.folder'
            }

            # pylint: disable=maybe-no-member
            file = service.files().create(body=file_metadata, fields='id'
                                          ).execute()
            print(F'Folder ID: "{file.get("id")}".')
            return file.get('id')

            
            
        except HttpError as error:
            # TODO(developer) - Handle errors from drive API.
            print(f'An error occurred: {error}')

                  
        
def createRemoteFolder(folderName, parentID):
    creds = None
    # The file token.json stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open('token.json', 'w') as token:
            token.write(creds.to_json())
            
    for items in folderName:
        
        drive_service = build('drive', 'v3', credentials=creds)

        # Create a folder on Drive, returns the newely created folders ID
        body = {
            'name': items,
            'mimeType': 'application/vnd.google-apps.folder',
            'parents':[parentID]

        }
    #     if parentID:
    #         body['parents'] = [{'id': parentID}]
        root_folder = drive_service.files().create(body = body).execute()
#     return root_folder['id']
        


# if __name__ == '__main__':
    
#     foldername='fifa_world_cup'
#     parentfolderID=create_parent_folder(foldername)
#     createRemoteFolder('amit',parentID=parentfolderID)
    


In [19]:
if __name__ == '__main__':
    content_name=content_details()
#     foldername='fifa_world_cup3'
    parentfolderID=create_parent_folder(content_name[0])
    creators_name=spreadsheet_record(content_name[0])
    createRemoteFolder(creators_name,parentID=parentfolderID)
    

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=563165058799-0ngsr5ll27hpp8ab6cbdhn7bifikk79t.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A55019%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&state=3WdnOPUGy0j5Wjm5NDIwWAqurMMv8j&access_type=offline
Folder ID: "1IrHTy9bcWTWfTLggT7dx3LrtlbUmTjBO".


#   Upload file from gdrive to GCP bucket

In [66]:

from __future__ import print_function

import os.path

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

# If modifying these scopes, delete the file token.json.
SCOPES = ['https://www.googleapis.com/auth/drive']



def get_blobs():
    client = storage.Client() 
    bucket_name = client.get_bucket("fifaworldcup2022")
    objects_in_bucket = client.list_blobs(bucket_name, fields='items(name)')
    object_names = []
    for object in objects_in_bucket:
        object_names.append(object.name)
    return object_names




def main(items):
    """Shows basic usage of the Drive v3 API.
    Prints the names and ids of the first 10 files the user has access to.
    """
    creds = None
    # The file token.json stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open('token.json', 'w') as token:
            token.write(creds.to_json())

    try:
        drive = build('drive', 'v3', credentials=creds)
        
        files = []
        page_token = None
        file_to_download = None
        objects_list = get_blobs()
        
       
        
        while True:
            
            mimtype="mimeType = 'application/vnd.google-apps.folder' and name = {}".format("'"+items+"'")
            folderId = drive.files().list(q = mimtype, pageSize=10, fields="nextPageToken, files(id, name)").execute()
            folderIdResult = folderId.get('files', [])

            id = folderIdResult[0].get('id')
            results = drive.files().list(q = "'" + id + "' in parents", pageSize=10, fields="nextPageToken, files(id, name)").execute()

            items = results.get('files', [])  

            if not items:
                print('No files found.')
                return
            print('Files:')
            for item in items:
                print(u'{0} ({1})'.format(item['name'], item['id']))
                file_id = item.get("id")
                file_name = item.get("name")
                if file_name not in objects_list:
                    request = drive.files().get_media(fileId=file_id)
                    file_download_buffer = io.BytesIO()
                    downloader = MediaIoBaseDownload(file_download_buffer, request)
                    done = False
                    while done is False:
                        status, done = downloader.next_chunk()
                        print(F'Download {int(status.progress() * 100)}.')

                    upload_to_drive(file_download_buffer,file_name)

            files.extend(folderId.get('files', []))
            page_token = folderId.get('nextPageToken', None)


            if page_token is None:
                break
         
            
            
    except HttpError as error:
        # TODO(developer) - Handle errors from drive API.
        print(f'An error occurred: {error}')

        
    
# def upload_to_drive(file, file_name):
#     client = storage.Client()
#     bucket_name = client.get_bucket("practise_bucket3456")
# #     blob = Blob(file_name, bucket_name)
#     blob = bucket_name.blob("amit/"+file_name)
#     #blob.upload_from_filename(source_file_name)
#     blob.upload_from_filename(file, rewind=True)
    
    
    
        
def upload_to_drive(file, file_name):
    client = storage.Client()
    bucket_name = client.get_bucket("fifaworldcup2022")
    blob = Blob(file_name, bucket_name)
    blob.upload_from_file(file, rewind=True)
    
    
    
    

if __name__ == '__main__':
    sub_folder=subfolder_name()
    for items in sub_folder:
        main(items)

Files:
suman (1ai0qZywfITvgY0TolRmOMEODQhCFuvOu)
sumit (148_MT8kjJqvBkOgcBLatHQcGIrF_u1rC)
amit (1FLDLu8vf8EiqQ2QNKIVs4-nv5kMtjBtC)
Files:
automation_requirement.txt (1NH5YrJBxuRe9KnSgWbNZK5gcaZbRUZF2)
Download 100.
Files:
bank.csv (1JmBqqqnCDLkxh-ZOqpr92uDDfLdtDi8j)
Download 100.
Files:
bank.csv (1vTHmGzWjz-WPKyrauyd8CeE3syOuMni7)
